In [1]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib


import Clone_Model
import rsc

In [2]:
tf.compat.v1.disable_eager_execution()

In [3]:
checkpoint_path = "./cloning_eager_dis/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning_eager_dis/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [30]:
#      CLONE MODEL EAGER DISABLED
###########################################
importlib.reload(rsc)
importlib.reload(Clone_Model)

K.clear_session() 

tf.compat.v1.disable_eager_execution()

batch_size = None #doesnt work/conflict tensorboard_callback
train, test = rsc.get_titanic_dataset()



model = rsc.get_model(None)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


tensorboard_callback.set_model(model)

history = model.fit(train,
                    batch_size=batch_size,
                    epochs=5, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])


"""
clone_fn = Clone_Model.clone_fn
clone = tf.keras.models.clone_model(model, input_tensors=model.inputs, clone_function=clone_fn)

z_in = np.ones(shape=(20,4))
assert np.allclose(model.predict(z_in), clone.predict(z_in)), "Wrooong"
"""
clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

history = clone.fit(train,
                    batch_size=batch_size,
                    epochs=1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[])
                

Train on 3 steps, validate on 1 steps
Epoch 1/5
3/3 [==============================] - 0s 25ms/step - loss: 1.5971 - val_loss: 1.2106
Epoch 2/5
3/3 [==============================] - 0s 6ms/step - loss: 1.2266 - val_loss: 1.7522
Epoch 3/5
3/3 [==============================] - 0s 11ms/step - loss: 1.4081 - val_loss: 0.7825
Epoch 4/5
3/3 [==============================] - 0s 7ms/step - loss: 1.6574 - val_loss: 0.6052
Epoch 5/5
3/3 [==============================] - 0s 3ms/step - loss: 1.0198 - val_loss: 0.6231
Train on 3 steps, validate on 1 steps
3/3 [==============================] - 0s 59ms/step - loss: 0.9545 - my_placeholder_metric_dense_1: 2534.1416 - observed_samples_sat_dense_1: 0.0000e+00 - my_placeholder_metric_dense_2: 763.4334 - observed_samples_sat_dense_2: 0.0000e+00 - my_placeholder_metric_dense_3: 645.1130 - observed_samples_sat_dense_3: 0.0000e+00 - my_placeholder_metric_dense_4: 388.9065 - observed_samples_sat_dense_4: 0.0000e+00 - my_placeholder_metric_predictions: 20